In [25]:
# Agora que você chegou aqui, faça o seguinte:
#1- Copie tudo que está aqui (Control A);
#2- Abra o Google Colab (https://colab.research.google.com/);
#3- Clique em New Notebook;
#4- Agora cole o script copiado. Já pode passar a ler no Google Colab (vai precisar logar no Google)!
#5- Clique no ícone "Ficheiros (ou Files)" no lado esquerdo e depois, acima clique no ícone para fazer o upload do arquivo.
#6- O arquivo vai ser carregado no Colab. Com o botão direito, copie o path do arquivo;
#7- Substitua o nome do arquivo onde está escrito 'entrada.xlsx' (linha 31)
#8- Execute o script clicando no botão play no canto superior no notebook.
#9- O script vai transformar os dados que estão em dias, horas minutos e segundos em hora decimal em uma outra coluna;
#10- Será gerado um arquivo de saída (saida.xlsx). Com o botão direiro, transfira para o notebook.
#11- Caso o arquivo não apareça, clique no botão atualizar.
#Dúvidas, entrar em contato com Marcelo Brum 21 98057-8932

import pandas as pd
import re

# Função para converter string de tempo para horas decimais
def time_to_decimal(time_str):
    if pd.isnull(time_str):  # Verifica se a string é NaN ou None
        return 0
    factors = {'d': 24*60, 'h': 60, 'm': 1, 's': 1/60}
    matches = re.findall(r'(\d+)([dhms])', time_str)
    return sum(int(value) * factors[unit] for value, unit in matches)

# Função para converter string de tempo para minutos
def time_to_minutes(time_str):
    return time_to_decimal(time_str)  # Como a função já retorna minutos, podemos reutilizá-la

# Carrega o arquivo Excel, ajustando a linha do cabeçalho caso necessário
df = pd.read_excel('entrada.xlsx', sheet_name='Sheet1', header=1)

# Verifica se a coluna "Duration" está presente
if 'Duration' not in df.columns:
    raise KeyError('A coluna "Duration" não foi encontrada no DataFrame.')

# Aplica a conversão à coluna "Duration"
df['Duration Decimal'] = df['Duration'].apply(lambda x: time_to_decimal(x) / 60)  # Converte para horas decimais
df['Duration Minutes'] = df['Duration'].apply(time_to_minutes)  # Converte para minutos

# Cria um DataFrame com o somatório do tempo por usuário
summary_df = df.groupby('User')['Duration Minutes'].sum().reset_index()

# Salva o DataFrame principal para um novo arquivo Excel
with pd.ExcelWriter('saida3.xlsx') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')

    # Salva o DataFrame de sumário em uma nova guia
    summary_df.to_excel(writer, index=False, sheet_name='Summary')

# O arquivo 'saida.xlsx' terá duas guias: 'Sheet1' com os dados originais e uma coluna adicional
# com o tempo em minutos e 'Summary' com o somatório do tempo por usuário.

In [28]:
import pandas as pd
import re

# Função para converter string de tempo para horas decimais
def time_to_decimal(time_str):
    if pd.isnull(time_str):  # Verifica se a string é NaN ou None
        return 0
    factors = {'d': 24*60, 'h': 60, 'm': 1, 's': 1/60}
    matches = re.findall(r'(\d+)([dhms])', time_str)
    return sum(int(value) * factors[unit] for value, unit in matches)

# Função para converter string de tempo para minutos
def time_to_minutes(time_str):
    return time_to_decimal(time_str)  # Como a função já retorna minutos, podemos reutilizá-la

# Carrega o arquivo Excel, ajustando a linha do cabeçalho caso necessário
df = pd.read_excel('entrada.xlsx', sheet_name='Sheet1', header=1)

# Verifica se a coluna "Duration" está presente
if 'Duration' not in df.columns:
    raise KeyError('A coluna "Duration" não foi encontrada no DataFrame.')

# Aplica a conversão à coluna "Duration"
df['Duration Minutes'] = df['Duration'].apply(time_to_minutes)  # Converte para minutos
df['Duration Decimal'] = df['Duration Minutes'] / 60  # Converte para horas decimais

# Cria um DataFrame com o somatório do tempo por usuário
summary_by_user = df.groupby('User')['Duration Minutes'].sum().reset_index()

# Cria um DataFrame com o somatório do tempo por estado (State)
summary_by_state = df.groupby('State')['Duration Minutes'].sum().reset_index()

# Separa o DataFrame por estados "FINISHED" e "KILLED"
finished_df = df[df['State'] == 'FINISHED']
killed_df = df[df['State'] == 'KILLED']

# Cria um DataFrame com o somatório do tempo por usuário para o estado "FINISHED"
summary_finished_by_user = finished_df.groupby('User')['Duration Minutes'].sum().reset_index()
summary_finished_by_user.rename(columns={'Duration Minutes': 'State Finished'}, inplace=True)

# Cria um DataFrame com o somatório do tempo por usuário para o estado "KILLED"
summary_killed_by_user = killed_df.groupby('User')['Duration Minutes'].sum().reset_index()
summary_killed_by_user.rename(columns={'Duration Minutes': 'State Killed'}, inplace=True)

# Junta os dois sumários em um único DataFrame
summary_by_user_and_state = pd.merge(summary_finished_by_user, summary_killed_by_user, on='User', how='outer').fillna(0)

# Salva o DataFrame principal e os DataFrames de sumário para um novo arquivo Excel
with pd.ExcelWriter('saida.xlsx') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    summary_by_user.to_excel(writer, index=False, sheet_name='Summary By User')
    summary_by_state.to_excel(writer, index=False, sheet_name='Summary By State')
    summary_by_user_and_state.to_excel(writer, index=False, sheet_name='Summary By User And State')

# O arquivo 'saida.xlsx' terá várias guias: 'Sheet1' com os dados originais e colunas adicionais,
# 'Summary By User' com o somatório do tempo por usuário,
# 'Summary By State' com o somatório do tempo por estado e
# 'Summary By User And State' com as colunas User, State Finished, State Killed e Duration Minutes.
